In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import nd2reader
import numpy as np
from cytoolz import compose
import skimage
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from paulssonlab.projects.molecule_counting.segmentation import segment, invert
from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
import itertools
import os.path
from os import path
import itertools

# import cv2
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import signal
import time

mpl.rcParams["figure.figsize"] = (10, 10)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
# Set up timers so that functions timeout and move on if something is weird and they don't run properly.
# Should diagnose this issue properly eventually, but for testing this should be ok.
class TimeoutException(Exception):  # Custom exception class
    pass


def timeout_handler(signum, frame):  # Custom signal handler
    raise TimeoutException


# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
# filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
#     "/n/scratch2/jqs1/200305/*.nd2")

filenames = glob.glob("/n/scratch3/users/j/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch3/users/j/jqs1/200305/*.nd2"
)

for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[2]
print("file", file)
nd2 = nd2reader.ND2Reader(file)
fov_index = 2
phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
# phase_img = nd2.get_frame_2D(v=0, c=0, z=0)[1100:3000,500:2250]
# fluor_img = nd2.get_frame_2D(v=0, c=1, z=0)[1100:3000,500:2250]
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img)
ax[1].imshow(fluor_img, cmap="RdGy")
# seg = segment_phase(phase_img)

In [ ]:
seg = segment_phase(phase_img)
fig, ax = plt.subplots()
# ax.imshow(seg)
ax.imshow(permute_labels(seg))
print(np.amax(seg))

In [ ]:
# Check segmented cell sizes
# unique, counts = np.unique(seg, return_counts=True)
# dict(zip(unique, counts))

In [ ]:
def get_indices_in_r(ref_indices, r):
    # ref_indices is a list of the indices that define the location of the reference point.
    # r is the radius within which we want to collect indices.
    ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
    rel_points = np.where(
        (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
    )
    points = [
        rel_points[0] + ref_indices[0] - np.ceil(r),
        rel_points[1] + ref_indices[1] - np.ceil(r),
    ]
    return points

In [ ]:
def rdf_single_ref_point_single_cell(cell, ref_point, time_allowed=80, start_r=1, dr=1):
    #     returns rdf for a single reference point in a single cell.
    #     cell is a masked array, where the mask is False where the cell is, and True everywhere else.
    #     cell.data is the fluorescence intensity array.
    #     ref_point is a tuple of 2 arrays, corresponding to the indices of the location of the 'nucleus'
    # rho is the average 'particle' density, where a particle corresponds to an intensity unit.
    rho = np.ma.mean(cell)
    #     print('cell sum in function',np.ma.sum(cell))
    r = start_r
    r_values = []
    #         masked_norm_fluor_img.mask is False where the cell is, and True outside.
    #         mask = np.copy(cell.mask)
    gr_values = []
    previous_r_vol = 0
    while np.count_nonzero(cell.mask == 0) > 0:
        #         print('points remaining',np.count_nonzero(cell.mask==0))
        #             While the cell mask still has 'False' values i.e. not all values have been used in the
        #             pair correlation function calculation somewhere.
        #             Get indices of the points within the current radius (r) of the ref point.
        indices_in_r = get_indices_in_r(ref_point, r)
        flat_indices_in_r = [item for sublist in indices_in_r for item in sublist]
        img_shape = np.shape(cell)
        #         Remove indices pairs which fall outside the image.
        if any(flat_indices_in_r >= np.min(img_shape)):
            indices_to_delete_0 = np.where(indices_in_r[0] >= img_shape[0])[0]
            indices_to_delete_1 = np.where(indices_in_r[1] >= img_shape[1])[0]
            indices_to_delete = np.concatenate(
                (indices_to_delete_0, indices_to_delete_1)
            )
            indices_to_delete = np.unique(indices_to_delete)
            indices_in_r[0] = np.delete(indices_in_r[0], indices_to_delete)
            indices_in_r[1] = np.delete(indices_in_r[1], indices_to_delete)
        # Calculate numerator (gr_num) and volume element (gr_vol)
        # gr_num should be the sum total of intensities of points in the current shell.
        #             Since the cell is masked, we don't need to worry about the non-cell regions.
        gr_num = np.sum(
            cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
        )
        #         print('grnum',gr_num)
        # gr_vol should be the number of points in the current shell.
        #             Note that the mask masks both the background and the previous regions that we have considered,
        #             so we simply subtract these from our list of indices in the current radius.
        gr_vol = len(indices_in_r[0]) - np.count_nonzero(
            cell.mask[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
        )
        #         print('gr vol',gr_vol)
        #             update the 'previous_r_vol' for the next radius shell.
        previous_r_vol = gr_vol
        #             Update the cell mask so that the values we have now included are not included
        #             in the next dr calculation.
        cell.mask[
            list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
        ] = True
        #             print('number of non masked values',np.count_nonzero(cell.mask==0))
        gr_values.append(gr_num / gr_vol)
        r_values.append(r)
        r += dr
    # Since rho is constant for all r, divide all at the same time to save on number of computations.
    gr_values = np.array(gr_values)
    gr_values = np.divide(gr_values, rho)
    return r_values, gr_values

In [ ]:
def rdf_multi_ref_point_single_cell(
    cell, ref_points, time_allowed=80, start_r=1, dr=1, intensity_weighted=True
):
    #     cell is a masked array, where the mask is False where the cell is, and True everywhere else.
    #     returns r_values, gr_values for one cell, where gr_values is the average gr over the
    #     ref_points.
    #     ref_points should be a tuple of 2 arrays, where the first array is the axis 0
    #     indices, and second array is the axis 1 indices, as we would get from np.where
    max_r_values = []
    all_gr_values = []
    if intensity_weighted:
        total_ref_point_intensity = np.sum(
            cell[list(map(int, ref_points[0])), list(map(int, ref_points[1]))]
        )
        num_ref_points = len(ref_points[0])
    #         total_cell_intensity = np.sum(cell)
    #         pixels_in_cell = np.count_nonzero(cell.mask == 0)
    for i in range(len(ref_points[0])):
        cell_for_analysis = np.ma.copy(cell)
        ref_point_0, ref_point_1 = ref_points[0][i], ref_points[1][i]
        r_values, gr_values = rdf_single_ref_point_single_cell(
            cell_for_analysis,
            (np.array(ref_point_0), np.array(ref_point_1)),
            time_allowed=80,
            start_r=start_r,
            dr=dr,
        )
        if intensity_weighted:
            gr_values *= cell.data[ref_point_0][ref_point_1]
        if len(r_values) > len(max_r_values):
            max_r_values = r_values
        all_gr_values.append(gr_values)
    gr_values_length_corrected = list(
        itertools.zip_longest(*all_gr_values, fillvalue=np.nan)
    )
    mean_gr_values = [np.nanmean(i) for i in gr_values_length_corrected]
    if intensity_weighted:
        #         mean_gr_values = np.multiply(pixels_in_cell,mean_gr_values/total_cell_intensity)
        mean_gr_values = np.multiply(
            num_ref_points, mean_gr_values / total_ref_point_intensity
        )
    return max_r_values, mean_gr_values

In [ ]:
def rdf(
    seg,
    fluor_img,
    ref_point_val="max",
    cell_indices=[0],
    time_allowed=80,
    savetxt="rdf",
    start_r=1,
    dr=1,
    intensity_weighted=True,
    stdevs=True,
    stdevs_savetxt="stdevs",
    fov_indices=[0],
    cell_norm="none",
):
    # Seg is the segmentation array.
    #     Get the background fluorescence of the image, to subtract from all cell fluorescence.
    min_fluor = np.amin(fluor_img)
    # Get number of cells in the image.
    cells = np.amax(seg)
    if cell_indices == "all":
        cell_indices = np.arange(cells)
    # all_r_values will be a list of lists of r values, while all_gr_values will be a list of lists, with
    # the corresponding g(r) (pair correlation function) values. The overall function will return
    # all_r_values, all_gr_values.
    all_r_values = []
    all_gr_values = []
    stdevs_list = []
    pixels = []
    for cell_index in cell_indices:
        if cell_index % 25 == 0:
            print("cell index", cell_index)
        if cell_index > cells:
            print("Error in rdf: cell_index larger than number of cells in fov.")
            pass
        #         Mask the fluorescence image everywhere except for the cell.
        cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
        # Normalise the intensities of the cell. The mean_int_cell isn't needed since everything gets normalised
        #         wrt total cell intensity anyway.
        #         mean_int_cell = np.mean(cell)
        cell -= min_fluor
        #         ref_point is the 'nucleus' i.e. the point relative to which the pair
        #         correlation function is to be calculated. For now, we set it to the
        #         maximum fluorescence intensity point of the cell.
        # CELL_NORM HERE
        if stdevs:
            stdevs_list.append(np.ma.std(cell))
        if ref_point_val == "max":
            ref_point = np.ma.where(cell == np.amax(cell))
            #         If there is more than one maximum fluorescence point, take the first one of these.
            if len(ref_point[0]) > 1:
                ref_point = (np.array([ref_point[0][0]]), np.array([ref_point[1][0]]))
        elif ref_point_val == "all":
            ref_point = np.where(cell.mask == 0)
        elif ref_point_val[0] == "sample":
            points_in_cell = np.where(cell.mask == 0)
            ref_point = (
                points_in_cell[0][:: ref_point_val[1]],
                points_in_cell[1][:: ref_point_val[1]],
            )
        elif (
            ref_point_val[0] == "max"
        ):  # ref_point_val=['max',10] to take mean of 10 most intense pixels
            cell_for_maxs = np.ma.copy(cell)
            ref_point = [[], []]
            for i in range(ref_point_val[1]):
                current_max = np.ma.where(cell_for_maxs == np.amax(cell_for_maxs))
                cell_for_maxs.mask[current_max] = 1
                ref_point[0].append(current_max[0][0])
                ref_point[1].append(current_max[1][0])
            ref_point = np.array(ref_point)
        else:
            ref_point = ref_point_val
        #         print('sum',np.ma.sum(cell))
        r_values, gr_values = rdf_multi_ref_point_single_cell(
            cell,
            ref_point,
            time_allowed=80,
            start_r=start_r,
            dr=dr,
            intensity_weighted=intensity_weighted,
        )
        #         print('int rdf',np.trapz(gr_values,x=r_values))
        #         print('sum gr',np.sum(gr_values))
        #         print('int rdf with 4pir^2',np.trapz(gr_values*np.pi*(2*np.array(r_values)+1),x=r_values))

        pixels.append(np.count_nonzero(cell.mask == 0))
        all_gr_values.append(gr_values)
        all_r_values.append(r_values)
    if savetxt:
        all_r_to_save = np.concatenate(all_r_values).ravel()
        all_gr_to_save = np.concatenate(all_gr_values).ravel()
        np.savetxt(savetxt, np.transpose(np.array([all_r_to_save, all_gr_to_save])))
    if stdevs_savetxt:
        np.savetxt(stdevs_savetxt, np.array(stdevs_list))
    return all_r_values, all_gr_values

In [ ]:
# Test using seg as the fluor image, since this should return g(r) = 1 for all r.
# Everything is 1 in this case
# r_values_list, gr_values_list = rdf(seg,seg,cell_indices=np.arange(6),
#                                     ref_point_val=['sample',10],dr=10,
#                                    intensity_weighted=True)
# Everything is also 1 in this case
# r_values_list, gr_values_list = rdf(seg,seg,cell_indices=np.arange(6),
#                                     ref_point_val=['sample',10],dr=10,
#                                    intensity_weighted=False)

# print('20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue')
# r,gr=rdf(seg,fluor_img,cell_indices='all',ref_point_val='max',dr=1,
#                                    intensity_weighted=True,
#                                    savetxt='20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue')

In [ ]:
def get_funcvals_from_txt(filename, start_r):
    # start_r is the start value of r in that dataset, so we can identify where each cell dataset starts.
    values = np.genfromtxt(filename)
    all_r_values = values[:, 0]
    all_func_values = values[:, 1]
    start_indices = np.where(all_r_values == start_r)[0]
    r_values_list = []
    func_values_list = []
    for i, val in enumerate(start_indices):
        if i != len(start_indices) - 1:
            r_values_list.append(all_r_values[val : start_indices[i + 1]])
            func_values_list.append(all_func_values[val : start_indices[i + 1]])
        else:
            r_values_list.append(all_r_values[val:])
            func_values_list.append(all_func_values[val:])
    return r_values_list, func_values_list

In [ ]:
# Function to plot gr from file
def plot_gr_from_file(
    filename,
    start_r,
    ax=None,
    savefig=None,
    indices_to_plot=None,
    legend=False,
    title=None,
    line=[1],
):
    r_values_list, gr_values_list = get_funcvals_from_txt(filename, start_r)
    if not ax:
        fig, ax = plt.subplots()
    if not indices_to_plot:
        indices_to_plot = np.arange(len(r_values_list))
    for i in indices_to_plot:
        ax.plot(r_values_list[i], gr_values_list[i])
    if legend:
        ax.legend([i for i in indices_to_plot])
    if title:
        ax.set_title(title)
    if line:
        for line_val in line:
            ax.axhline(y=line_val, ls=":", color="k")
    if savefig:
        fig.savefig(savefig)

In [ ]:
# Autocorrelation function
# Can do multiple cells in same image
def autocorr(
    seg,
    fluor_img,
    cell_indices=[0],
    time_allowed=80,
    savetxt="autocorr",
    ref_point_val="max",
    r_values=None,
    gr_values=None,
    intensity_weighted=True,
    start_r=1,
    dr=1,
    rdfsavetxt="rdf",
    stdevs=True,
    stdevs_savetxt="stdevs",
):
    if not r_values or not gr_values:
        r_values, gr_values = rdf(
            seg,
            fluor_img,
            ref_point_val=ref_point_val,
            cell_indices=cell_indices,
            time_allowed=80,
            savetxt=rdfsavetxt,
            start_r=start_r,
            dr=dr,
            intensity_weighted=intensity_weighted,
            stdevs=stdevs,
            stdevs_savetxt=stdevs_savetxt,
        )
    if cell_indices == "all":
        cell_indices = np.arange(np.amax(seg))
    for cell_index in cell_indices:
        min_fluor = np.amin(fluor_img)
        cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
        cell -= min_fluor
    all_auto_corr = []
    all_k_values = []
    for i, cell_index in enumerate(cell_indices):
        r_vals = r_values[i]
        gr_vals = gr_values[i]
        gr_mean = np.mean(gr_values[i])
        k_vals = np.arange(dr, r_vals[-1], dr)
        all_k_values.append(k_vals)
        auto_corr = []
        denom = 0
        for i in range(len(r_vals)):
            denom += np.power(gr_vals[i] - gr_mean, 2)
        for ki, k in enumerate(k_vals):
            num = 0
            contrib_to_num = 0
            for i in k_vals[: len(k_vals) - ki]:
                contrib_to_num += 1
                num += (gr_vals[i - 1] - gr_mean) * (gr_vals[i + ki] - gr_mean)
            num = num / contrib_to_num
            auto_corr.append(num / denom)
        all_auto_corr.append(auto_corr)
    if savetxt:
        all_r_to_save = np.concatenate(
            np.array([r_vals[:-1] for r_vals in r_values])
        ).ravel()
        all_autocorr_to_save = np.concatenate(all_auto_corr).ravel()
        np.savetxt(
            savetxt, np.transpose(np.array([all_r_to_save, all_autocorr_to_save]))
        )
    return all_k_values, all_auto_corr

In [ ]:
# Initial gradient of rdf or autocorr might not be that good - if we have a random high intensity point,
# it will very quickly drop since it's not a real blob, so this would increase the gradient artificially.
# Instead, calculate area under curve before autocorr = 0 or rdf = 1.

# def gradients(r_vals, func_vals, savetxt = 'gradients'):
# # r_vals is a list of lists of r_values, and func_vals is a list of lists of function values
#     all_gradient_vals = []
#     for i,r_val_list in enumerate(r_vals):
#         gradient_vals = np.gradient(func_vals[i], r_val_list)
#         all_gradient_vals.append(gradient_vals)
#     if savetxt:

#     return all_gradient_vals

In [ ]:
def area_under(
    r_vals,
    func_vals,
    threshold,
    as_frac_total=False,
    shift=None,
    below_thresh_zero=False,
    as_frac_total_original=False,
    total_area_above_thresh=False,
):
    #     threshold = area above the first crossing of which we want to integrate.
    # as_frac_total = True means our output will be the above area divided by the total integral of the shifted function.
    # shift = if we want to integrate RDF-1, then shift = 1
    # below_thresh_zero = True means all values below the threshold (of the shifted function)
    # will be set to 0 and so be ignore in the integration.
    # as_frac_total_original = True means our output will be the above area divided
    # by the total integral of the UNshifted function.
    # total_area_above_thresh = True means instead of just integrating above the first cross,
    # we integrate everything above the threshold.
    func_vals_original = np.copy(func_vals)
    if shift:
        func_vals -= shift
    if not total_area_above_thresh:
        i = 0
        while i < len(func_vals) and func_vals[i] > threshold:
            i += 1
        r_vals_thresh, func_vals_thresh = r_vals[:i], func_vals[:i]
        area = np.trapz(func_vals_thresh, x=r_vals_thresh)
    else:
        f_vals = np.copy(func_vals)
        for i, val in enumerate(f_vals):
            if val < threshold:
                f_vals[i] = 0
        area = np.trapz(f_vals, x=r_vals)
    if below_thresh_zero:
        for i, val in enumerate(func_vals):
            if val < threshold:
                func_vals[i] = 0
    if as_frac_total:
        if as_frac_total_original:
            total_int = np.trapz(func_vals_original, x=r_vals)
        else:
            total_int = np.trapz(func_vals, x=r_vals)
        area /= total_int
    return area

In [ ]:
def area_under_from_file(
    filename,
    start_r,
    threshold,
    as_frac_total=False,
    shift=None,
    below_thresh_zero=False,
    as_frac_total_original=False,
    total_area_above_thresh=False,
):
    r_values_list, func_values_list = get_funcvals_from_txt(filename, start_r)
    area_under_list = []
    for i in range(len(r_values_list)):
        area_under_list.append(
            area_under(
                r_values_list[i],
                func_values_list[i],
                threshold,
                as_frac_total=as_frac_total,
                shift=shift,
                below_thresh_zero=below_thresh_zero,
                as_frac_total_original=as_frac_total_original,
                total_area_above_thresh=total_area_above_thresh,
            )
        )
    return area_under_list

In [ ]:
def first_threshold_cross(r_vals, func_vals, threshold, shift=None):
    if shift:
        func_vals -= shift
    i = 0
    while i < len(func_vals) and func_vals[i] > threshold:
        i += 1
    return r_vals[i - 1]

In [ ]:
def first_threshold_cross_from_file(filename, start_r, threshold, shift=None):
    r_values_list, func_values_list = get_funcvals_from_txt(filename, start_r)
    results_list = []
    for i in range(len(r_values_list)):
        results_list.append(
            first_threshold_cross(
                r_values_list[i], func_values_list[i], threshold, shift=shift
            )
        )
    return results_list

In [ ]:
def number_threshold_crosses(
    r_vals, func_vals, threshold, shift=None, rel_to_length=False
):
    if shift:
        func_vals -= shift
    crosses_count = 0
    for i in range(len(r_vals) - 1):
        if func_vals[i] <= threshold and func_vals[i + 1] > threshold:
            crosses_count += 1
        elif func_vals[i] > threshold and func_vals[i + 1] <= threshold:
            crosses_count += 1
    if rel_to_length:
        crosses_count /= len(r_vals)
    return crosses_count

In [ ]:
def number_threshold_crosses_from_file(
    filename, start_r, threshold, shift=None, rel_to_length=False
):
    r_values_list, func_values_list = get_funcvals_from_txt(filename, start_r)
    results_list = []
    for i in range(len(r_values_list)):
        results_list.append(
            number_threshold_crosses(
                r_values_list[i],
                func_values_list[i],
                threshold,
                shift=shift,
                rel_to_length=rel_to_length,
            )
        )
    return results_list

In [ ]:
def expected_func_val(r_vals, func_vals):
    inside = np.multiply(r_vals, func_vals)
    expected_val = np.trapz(inside, x=r_vals)
    norm_factor = np.trapz(func_vals, x=r_vals)
    expected_val /= norm_factor
    return expected_val

In [ ]:
def expected_func_val_from_file(filename, start_r):
    r_values_list, func_values_list = get_funcvals_from_txt(filename, start_r)
    expected_val_list = []
    for i in range(len(r_values_list)):
        expected_val_list.append(
            expected_func_val(r_values_list[i], func_values_list[i])
        )
    return expected_val_list

In [ ]:
def get_norm_func_from_file(filename, start_r, savetxt=None):
    r_values_list, func_values_list = get_funcvals_from_txt(filename, start_r)
    norm_func_values_list = []
    for i, r_vals in enumerate(r_values_list):
        norm_factor = np.trapz(func_values_list[i], x=r_vals)
        norm_func_values_list.append(func_values_list[i] / norm_factor)
    if savetxt:
        all_r_to_save = np.concatenate(r_values_list).ravel()
        all_norm_func_to_save = np.concatenate(norm_func_values_list).ravel()
        np.savetxt(
            savetxt, np.transpose(np.array([all_r_to_save, all_norm_func_to_save]))
        )
    return r_values_list, norm_func_values_list

In [ ]:
# def smooth_func_data_from_file(filename, start_r, smooth_window):
#     r_values_list, func_values_list = get_funcvals_from_txt(filename,start_r)
#     for i,r_vals in enumerate(r_values_list):


#     def moving_average(a, n=3) :
#     ret = np.cumsum(a, dtype=float)
#     ret[n:] = ret[n:] - ret[:-n]
#     return ret[n - 1:] / n

In [ ]:
files_to_analyse = [
    [14, "chr_P_sfGFP_100x_ref5max", 20, "all", 5],
    [3, "chr_P_GFPmut3_100x_ref5max", 50, "all", 5],
    [16, "chr_P_sfGFP_40x_ref5max", 10, np.arange(400), 5],
    [6, "chr_P_GFPmut3_40x_ref5max", 0, np.arange(400), 5],
    [15, "chr_P_sfGFP_20x_ref5max", 0, np.arange(400), 5],
    [4, "chr_P_GFPmut3_20x_ref5max", 0, np.arange(400), 5],
    [14, "chr_P_sfGFP_100x_ref10max", 20, "all", 10],
    [3, "chr_P_GFPmut3_100x_ref10max", 50, "all", 10],
    [16, "chr_P_sfGFP_40x_ref10max", 10, np.arange(200), 10],
    [6, "chr_P_GFPmut3_40x_ref10max", 0, np.arange(200), 10],
    [15, "chr_P_sfGFP_20x_ref10max", 0, np.arange(200), 10],
    [4, "chr_P_GFPmut3_20x_ref10max", 0, np.arange(200), 10],
]
# files_to_analyse = [[16,"chr_P_sfGFP_40x_ref5max",10,np.arange()],[6,"chr_P_GFPmut3_40x_ref5max",0]]
# files_to_analyse = [[15,"chr_P_sfGFP_20x_ref1max",0,'all'],[4,"chr_P_GFPmut3_20x_ref1max",0,'all']]
calculate = True
date_stamp_data = "20200729"
date_stamp_output = "20200729"


for file in files_to_analyse:
    nd2 = nd2reader.ND2Reader(filenames[file[0]])
    fov_index = file[2]
    name = file[1]
    if not path.exists(date_stamp_data + "/autocorr_" + name) or calculate:
        print("Calculating")
        phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
        fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
        seg = segment_phase(phase_img)
        autocorr(
            seg,
            fluor_img,
            cell_indices=file[3],
            time_allowed=80,
            savetxt=date_stamp_output + "/autocorr_" + name,
            ref_point_val=["max", file[4]],
            r_values=None,
            gr_values=None,
            intensity_weighted=True,
            start_r=1,
            dr=1,
            rdfsavetxt=date_stamp_output + "/rdf_" + name,
            stdevs=True,
            stdevs_savetxt=date_stamp_output + "/stdevs_" + name,
        )

    fig, axes = plt.subplots(1, 5, figsize=(12, 3))
    ax = axes.ravel()
    plot_gr_from_file(
        date_stamp_data + "/rdf_" + name, 1, savefig=None, line=[1], ax=ax[0]
    )
    ax[0].set_title("RDF")

    area_under_rdf = area_under_from_file(date_stamp_data + "/rdf_" + name, 1, 1)
    ax[1].hist(area_under_rdf, bins=20, range=(0, 40))
    ax[1].set_title("Integral RDF>1")

    area_under_rdf_frac = area_under_from_file(
        date_stamp_data + "/rdf_" + name, 1, 1, as_frac_total=True
    )
    ax[2].hist(area_under_rdf_frac, bins=20, range=(0, 1))
    ax[2].set_title("Integral RDF>1 as frac")

    expected_rdfs = expected_func_val_from_file(date_stamp_data + "/rdf_" + name, 1)
    ax[3].hist(expected_rdfs, bins=20, range=(0, 40))
    ax[3].set_title("Mean RDF")

    stdevs = np.genfromtxt(date_stamp_output + "/stdevs_" + name)
    ax[4].hist(stdevs, bins=20)
    ax[4].set_title("Stdevs of cell intensities")

    fig.suptitle(name + " fov: " + str(fov_index), fontsize=14, y=1.05)
    plt.tight_layout()
    fig.savefig(date_stamp_output + "/plots_" + name, bbox_inches="tight")

In [ ]:
def generate_analysis_plots(files_to_analyse):
    #     files_to_analyse is a list of lists.
    # Each list contains the information about the file, in the order:
    # [filename_index, base_filename, fov (rewrite in future to do multiple), date_stamp_data, date_stamp_output,
    # ref_points, cell_indices, calculate (true if want to re-calculate, otherwise will check for existing data files)]
    for file_details in files_to_analyse:
        [
            file_index,
            name,
            fov_index,
            date_stamp_data,
            date_stamp_output,
            ref_points,
            cell_indices,
            calculate,
        ] = file_details
        nd2 = nd2reader.ND2Reader(filenames[file_index])
        seg_available = False
        if not path.exists(date_stamp_data + "/autocorr_" + name) or calculate:
            print("Calculating")
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            signal.alarm(120)
            try:
                seg = segment_phase(phase_img)
                seg_available = True
            except TimeoutException:
                print("timeout")
                continue
            signal.alarm(0)
            autocorr(
                seg,
                fluor_img,
                cell_indices=cell_indices,
                time_allowed=80,
                savetxt=date_stamp_output + "/autocorr_" + name,
                ref_point_val=["max", ref_points],
                r_values=None,
                gr_values=None,
                intensity_weighted=True,
                start_r=1,
                dr=1,
                rdfsavetxt=date_stamp_output + "/rdf_" + name,
                stdevs=True,
                stdevs_savetxt=date_stamp_output + "/stdevs_" + name,
            )
        fig, axes = plt.subplots(6, 4, figsize=(12, 18))
        ax = axes.ravel()
        plot_gr_from_file(
            date_stamp_data + "/rdf_" + name, 1, savefig=None, line=[1], ax=ax[0]
        )
        ax[0].set_title("RDF")

        val_first_rdf_cross = first_threshold_cross_from_file(
            date_stamp_data + "/rdf_" + name, 1, 0, shift=1
        )
        area_under_rdf_shift1_thresh0 = area_under_from_file(
            date_stamp_data + "/rdf_" + name,
            1,
            0,
            shift=1,
            as_frac_total=False,
            below_thresh_zero=False,
        )
        area_under_rdf_shift1_thresh0_as_frac_total = area_under_from_file(
            date_stamp_data + "/rdf_" + name,
            1,
            0,
            shift=1,
            as_frac_total=True,
            below_thresh_zero=True,
            as_frac_total_original=True,
        )
        number_rdf_crosses = number_threshold_crosses_from_file(
            date_stamp_data + "/rdf_" + name, 1, 0, shift=1, rel_to_length=True
        )
        total_area_under_rdf_shift1_thresh0_as_frac_total = area_under_from_file(
            date_stamp_data + "/rdf_" + name,
            1,
            0,
            shift=1,
            as_frac_total=True,
            below_thresh_zero=True,
            as_frac_total_original=True,
            total_area_above_thresh=True,
        )

        stdevs = np.genfromtxt(date_stamp_output + "/stdevs_" + name)

        to_plot = [
            ["First RDF-1=0", val_first_rdf_cross, 20],
            ["Initial integral RDF-1>0", area_under_rdf_shift1_thresh0, 25],
            [
                "Initial integral RDF-1>0 as frac total",
                area_under_rdf_shift1_thresh0_as_frac_total,
                1,
            ],
            ["RDF-1 fluctuation frequency", number_rdf_crosses, 1],
            [
                "Total integral RDF-1>0 as frac total",
                total_area_under_rdf_shift1_thresh0_as_frac_total,
                1,
            ],
            ["Stdevs of cell intensity", stdevs],
        ]

        for i in np.arange(1, 7):
            if len(to_plot[i - 1]) > 2:
                ax[i].hist(to_plot[i - 1][1], bins=20, range=(0, to_plot[i - 1][2]))
            else:
                ax[i].hist(to_plot[i - 1][1], bins=20)
            ax[i].set_title(to_plot[i - 1][0])

        combs = itertools.combinations([0, 1, 2, 3, 4, 5], 2)

        for i, comb in enumerate(combs):
            ax[i + 7].scatter(to_plot[comb[0]][1], to_plot[comb[1]][1])
            ax[i + 7].set_xlabel(to_plot[comb[0]][0])
            ax[i + 7].set_ylabel(to_plot[comb[1]][0])
            if len(to_plot[comb[0]]) > 2:
                ax[i + 7].set_xlim(0, to_plot[comb[0]][2])
            if len(to_plot[comb[1]]) > 2:
                ax[i + 7].set_ylim(0, to_plot[comb[1]][2])

        if seg_available:
            ax[-1].imshow(seg)

        fig.suptitle(name + " fov: " + str(fov_index), fontsize=14, y=1.05)
        plt.tight_layout()
        fig.savefig(date_stamp_output + "/plots_" + name, bbox_inches="tight")

In [ ]:
def plot_stdev_rdffrac_overlay(
    stdev_files,
    rdf_files,
    legend_titles,
    savefig,
    plot_title,
    stdev_lim=None,
    rdf_frac_lim=None,
    all_linlog=False,
    axes=None,
    fig=None,
):
    if all_linlog:
        fig, axes = plt.subplots(1, 4, figsize=(12, 3))
        ax = axes.ravel()
    else:
        if not axes and not fig:
            fig, axes = plt.subplots(figsize=(6, 6))
    colour = 0
    for i, file in enumerate(stdev_files):
        stdevs = np.genfromtxt(file)
        rdf_fracs = area_under_from_file(
            rdf_files[i],
            1,
            0,
            shift=1,
            as_frac_total=True,
            below_thresh_zero=True,
            as_frac_total_original=True,
        )
        edge_colours = [
            "xkcd:blue blue",
            "xkcd:yellow orange",
            "xkcd:lightish red",
            "xkcd:grassy green",
        ]
        if all_linlog:
            for panel in ax:
                panel.scatter(
                    stdevs,
                    rdf_fracs,
                    label=legend_titles[i],
                    facecolors="none",
                    edgecolors=edge_colours[colour],
                    linewidths=0.75,
                    alpha=0.8,
                )
                panel.set_xlabel("Stdevs of cell intensity")
                panel.set_ylabel("Initial integral RDF-1>0 as frac total")
        else:
            axes.scatter(
                stdevs,
                rdf_fracs,
                label=legend_titles[i],
                facecolors="none",
                edgecolors=edge_colours[colour],
                linewidths=0.75,
                alpha=0.8,
            )
            axes.set_xlabel("Stdevs of cell intensity")
            axes.set_ylabel("Initial integral RDF-1>0 as frac total")
        colour += 1
    if all_linlog:
        ax[1].set_xscale("log")
        ax[2].set_yscale("log")
        ax[2].set_ylim(ymin=5e-3, ymax=1)
        ax[3].set_xscale("log")
        ax[3].set_yscale("log")
        ax[3].set_ylim(ymin=5e-3, ymax=1)
    else:
        axes.set_xscale("log")
        axes.set_yscale("log")
        axes.set_ylim(ymin=5e-3, ymax=1)
    plt.legend()
    fig.suptitle(plot_title, fontsize=14, y=1.05)
    plt.tight_layout()
    fig.savefig(savefig, bbox_inches="tight")

In [ ]:
plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_100x_ref1max",
        "20200803/stdevs_chr_P_Dendra2_100x_ref1max",
        "20200729/stdevs_chr_P_sfGFP_100x_ref1max",
        "20200729/stdevs_chr_P_GFPmut3_100x_ref1max",
    ],
    [
        "20200803/rdf_chr_P_Venus_100x_ref1max",
        "20200803/rdf_chr_P_Dendra2_100x_ref1max",
        "20200729/rdf_chr_P_sfGFP_100x_ref1max",
        "20200729/rdf_chr_P_GFPmut3_100x_ref1max",
    ],
    ["100x Venus ref1", "100x Dendra2 ref1", "100x sfGFP ref1", "100x GFPmut3 ref1"],
    "20200803/all_100x_ref1",
    "100x ref1",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_40x_ref1max",
        "20200803/stdevs_chr_P_Dendra2_40x_ref1max",
        "20200729/stdevs_chr_P_sfGFP_40x_ref1max",
        "20200729/stdevs_chr_P_GFPmut3_40x_ref1max",
    ],
    [
        "20200803/rdf_chr_P_Venus_40x_ref1max",
        "20200803/rdf_chr_P_Dendra2_40x_ref1max",
        "20200729/rdf_chr_P_sfGFP_40x_ref1max",
        "20200729/rdf_chr_P_GFPmut3_40x_ref1max",
    ],
    ["40x Venus ref1", "40x Dendra2 ref1", "40x sfGFP ref1", "40x GFPmut3 ref1"],
    "20200803/all_40x_ref1",
    "40x ref1",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_20x_ref1max",
        "20200803/stdevs_chr_P_Dendra2_20x_ref1max",
        "20200729/stdevs_chr_P_sfGFP_20x_ref1max",
        "20200729/stdevs_chr_P_GFPmut3_20x_ref1max",
    ],
    [
        "20200803/rdf_chr_P_Venus_20x_ref1max",
        "20200803/rdf_chr_P_Dendra2_20x_ref1max",
        "20200729/rdf_chr_P_sfGFP_20x_ref1max",
        "20200729/rdf_chr_P_GFPmut3_20x_ref1max",
    ],
    ["20x Venus ref1", "20x Dendra2 ref1", "20x sfGFP ref1", "20x GFPmut3 ref1"],
    "20200803/all_20x_ref1",
    "20x ref1",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_100x_ref5max",
        "20200803/stdevs_chr_P_Dendra2_100x_ref5max",
        "20200729/stdevs_chr_P_sfGFP_100x_ref5max",
        "20200729/stdevs_chr_P_GFPmut3_100x_ref5max",
    ],
    [
        "20200803/rdf_chr_P_Venus_100x_ref5max",
        "20200803/rdf_chr_P_Dendra2_100x_ref5max",
        "20200729/rdf_chr_P_sfGFP_100x_ref5max",
        "20200729/rdf_chr_P_GFPmut3_100x_ref5max",
    ],
    ["100x Venus ref5", "100x Dendra2 ref5", "100x sfGFP ref5", "100x GFPmut3 ref5"],
    "20200803/all_100x_ref5",
    "100x ref5",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_40x_ref5max",
        "20200803/stdevs_chr_P_Dendra2_40x_ref5max",
        "20200729/stdevs_chr_P_sfGFP_40x_ref5max",
        "20200729/stdevs_chr_P_GFPmut3_40x_ref5max",
    ],
    [
        "20200803/rdf_chr_P_Venus_40x_ref5max",
        "20200803/rdf_chr_P_Dendra2_40x_ref5max",
        "20200729/rdf_chr_P_sfGFP_40x_ref5max",
        "20200729/rdf_chr_P_GFPmut3_40x_ref5max",
    ],
    ["40x Venus ref5", "40x Dendra2 ref5", "40x sfGFP ref5", "40x GFPmut3 ref5"],
    "20200803/all_40x_ref5",
    "40x ref5",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_20x_ref5max",
        "20200803/stdevs_chr_P_Dendra2_20x_ref5max",
        "20200729/stdevs_chr_P_sfGFP_20x_ref5max",
        "20200729/stdevs_chr_P_GFPmut3_20x_ref5max",
    ],
    [
        "20200803/rdf_chr_P_Venus_20x_ref5max",
        "20200803/rdf_chr_P_Dendra2_20x_ref5max",
        "20200729/rdf_chr_P_sfGFP_20x_ref5max",
        "20200729/rdf_chr_P_GFPmut3_20x_ref5max",
    ],
    ["20x Venus ref5", "20x Dendra2 ref5", "20x sfGFP ref5", "20x GFPmut3 ref5"],
    "20200803/all_20x_ref5",
    "20x ref5",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_100x_ref10max",
        "20200803/stdevs_chr_P_Dendra2_100x_ref10max",
        "20200729/stdevs_chr_P_sfGFP_100x_ref10max",
        "20200729/stdevs_chr_P_GFPmut3_100x_ref10max",
    ],
    [
        "20200803/rdf_chr_P_Venus_100x_ref10max",
        "20200803/rdf_chr_P_Dendra2_100x_ref10max",
        "20200729/rdf_chr_P_sfGFP_100x_ref10max",
        "20200729/rdf_chr_P_GFPmut3_100x_ref10max",
    ],
    [
        "100x Venus ref10",
        "100x Dendra2 ref10",
        "100x sfGFP ref10",
        "100x GFPmut3 ref10",
    ],
    "20200803/all_100x_ref10",
    "100x ref10",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_40x_ref10max",
        "20200803/stdevs_chr_P_Dendra2_40x_ref10max",
        "20200729/stdevs_chr_P_sfGFP_40x_ref10max",
        "20200729/stdevs_chr_P_GFPmut3_40x_ref10max",
    ],
    [
        "20200803/rdf_chr_P_Venus_40x_ref10max",
        "20200803/rdf_chr_P_Dendra2_40x_ref10max",
        "20200729/rdf_chr_P_sfGFP_40x_ref10max",
        "20200729/rdf_chr_P_GFPmut3_40x_ref10max",
    ],
    ["40x Venus ref10", "40x Dendra2 ref10", "40x sfGFP ref10", "40x GFPmut3 ref10"],
    "20200803/all_40x_ref10",
    "40x ref10",
)

plot_stdev_rdffrac_overlay(
    [
        "20200803/stdevs_chr_P_Venus_20x_ref10max",
        "20200803/stdevs_chr_P_Dendra2_20x_ref10max",
        "20200729/stdevs_chr_P_sfGFP_20x_ref10max",
        "20200729/stdevs_chr_P_GFPmut3_20x_ref10max",
    ],
    [
        "20200803/rdf_chr_P_Venus_20x_ref10max",
        "20200803/rdf_chr_P_Dendra2_20x_ref10max",
        "20200729/rdf_chr_P_sfGFP_20x_ref10max",
        "20200729/rdf_chr_P_GFPmut3_20x_ref10max",
    ],
    ["20x Venus ref10", "20x Dendra2 ref10", "20x sfGFP ref10", "20x GFPmut3 ref10"],
    "20200803/all_20x_ref10",
    "20x ref10",
)

In [ ]:
# 20200803
fig, ax = plt.subplots()
plot_stdev_rdffrac_overlay(
    [
        "20200729/stdevs_chr_P_sfGFP_20x_ref1max",
        "20200729/stdevs_chr_P_GFPmut3_20x_ref1max",
    ],
    ["20200729/rdf_chr_P_sfGFP_20x_ref1max", "20200729/rdf_chr_P_GFPmut3_20x_ref1max"],
    ["20x sfGFP ref1", "20x GFPmut3 ref1"],
    "20200803/sf_mut3_20x_ref1",
    "sfGFP and GFPmut3 20x ref1",
    axes=ax,
    fig=fig,
)
ax.axhline(y=1.5e-1, ls=":", color="k")
ax.axvline(x=60, ls=":", color="k")
ax.axvline(x=200, ls=":", color="k")
ax.axvline(x=1000, ls=":", color="k")

fig.savefig("20200803/sf_mut3_20x_ref1")

stdevs_sfGFP = np.genfromtxt("20200729/stdevs_chr_P_sfGFP_20x_ref1max")
stdevs_mut3 = np.genfromtxt("20200729/stdevs_chr_P_GFPmut3_20x_ref1max")
ints_sfGFP = area_under_from_file(
    "20200729/rdf_chr_P_sfGFP_20x_ref1max",
    1,
    0,
    shift=1,
    as_frac_total=True,
    below_thresh_zero=True,
    as_frac_total_original=True,
)
ints_mut3 = area_under_from_file(
    "20200729/rdf_chr_P_GFPmut3_20x_ref1max",
    1,
    0,
    shift=1,
    as_frac_total=True,
    below_thresh_zero=True,
    as_frac_total_original=True,
)

A_sfGFP = []
B_sfGFP = []
C_sfGFP = []
D_sfGFP = []
E_sfGFP = []

A_mut3 = []
B_mut3 = []
C_mut3 = []
D_mut3 = []
E_mut3 = []

for i, stdev in enumerate(stdevs_sfGFP):
    if stdev <= 60:
        A_sfGFP.append(i)
    elif stdev > 60 and stdev <= 200:
        if ints_sfGFP[i] < 1.5e-1:
            B_sfGFP.append(i)
        else:
            C_sfGFP.append(i)
    elif stdev > 200 and stdev <= 1000:
        D_sfGFP.append(i)
    elif stdev > 1000:
        E_sfGFP.append(i)

for i, stdev in enumerate(stdevs_mut3):
    if stdev <= 60:
        A_mut3.append(i)
    elif stdev > 60 and stdev <= 200:
        if ints_mut3[i] < 1.5e-1:
            B_mut3.append(i)
        else:
            C_mut3.append(i)
    elif stdev > 200 and stdev <= 1000:
        D_mut3.append(i)
    elif stdev > 1000:
        E_mut3.append(i)

In [ ]:
sfGFP_file = filenames[15]
sfGFP_img = nd2reader.ND2Reader(sfGFP_file)
fov_index = 0
sfGFP_phase_img = sfGFP_img.get_frame_2D(v=0, c=0, z=fov_index)
sfGFP_fluor_img = sfGFP_img.get_frame_2D(v=0, c=1, z=fov_index)
sfGFP_seg = segment_phase(sfGFP_phase_img)

mut3_file = filenames[4]
mut3_img = nd2reader.ND2Reader(mut3_file)
fov_index = 0
mut3_phase_img = mut3_img.get_frame_2D(v=0, c=0, z=fov_index)
mut3_fluor_img = mut3_img.get_frame_2D(v=0, c=1, z=fov_index)
mut3_seg = segment_phase(mut3_phase_img)

fig, axes = plt.subplots(1, 2)
ax = axes.ravel()
ax[0].imshow(sfGFP_seg)
ax[1].imshow(mut3_seg)

In [ ]:
# Get images of cells from different regions of the 2d stdevs/frac RDF integrals for sfGFP and GFPmut3.
columns = 10

fig, axes = plt.subplots(7, columns, figsize=(columns * 3, 21))
ax = axes.ravel()
i = 0
for plot in np.arange(0, columns):
    cell_index = A_mut3[plot]
    cell_indices = np.where(mut3_seg == cell_index + 1)
    cell = np.ma.masked_where(mut3_seg != cell_index + 1, mut3_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == 0:
        ax[plot].set_title("GFPmut3 class A \n" + str(cell_index))

for plot in np.arange(columns, columns * 2):
    cell_index = A_sfGFP[plot - columns]
    cell_indices = np.where(sfGFP_seg == cell_index + 1)
    cell = np.ma.masked_where(sfGFP_seg != cell_index + 1, sfGFP_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns:
        ax[plot].set_title("sfGFP class A \n" + str(cell_index))

for plot in np.arange(columns * 2, columns * 3):
    try:
        cell_index = B_mut3[plot - columns * 2]
    except IndexError:
        break
    cell_indices = np.where(mut3_seg == cell_index + 1)
    cell = np.ma.masked_where(mut3_seg != cell_index + 1, mut3_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns * 2:
        ax[plot].set_title("GFPmut3 class B \n" + str(cell_index))

for plot in np.arange(columns * 3, columns * 4):
    cell_index = B_sfGFP[plot - columns * 3]
    cell_indices = np.where(sfGFP_seg == cell_index + 1)
    cell = np.ma.masked_where(sfGFP_seg != cell_index + 1, sfGFP_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns * 3:
        ax[plot].set_title("sfGFP class B \n" + str(cell_index))

for plot in np.arange(columns * 4, columns * 5):
    cell_index = C_sfGFP[plot - columns * 4]
    cell_indices = np.where(sfGFP_seg == cell_index + 1)
    cell = np.ma.masked_where(sfGFP_seg != cell_index + 1, sfGFP_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns * 4:
        ax[plot].set_title("sfGFP class C \n" + str(cell_index))

for plot in np.arange(columns * 5, columns * 6):
    cell_index = D_sfGFP[plot - columns * 5]
    cell_indices = np.where(sfGFP_seg == cell_index + 1)
    cell = np.ma.masked_where(sfGFP_seg != cell_index + 1, sfGFP_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns * 5:
        ax[plot].set_title("sfGFP class D \n" + str(cell_index))

for plot in np.arange(columns * 6, columns * 7):
    try:
        cell_index = E_sfGFP[plot - columns * 6]
    except IndexError:
        break
    cell_indices = np.where(sfGFP_seg == cell_index + 1)
    cell = np.ma.masked_where(sfGFP_seg != cell_index + 1, sfGFP_fluor_img)
    ax[plot].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[plot].set_title(cell_index)
    if plot == columns * 6:
        ax[plot].set_title("sfGFP class E \n" + str(cell_index))

plt.tight_layout()
fig.savefig("20200803/sf_mut3_20xref1_cellsAtoE", bbox_inches="tight")

In [ ]:
# files_to_analyse = [[14,"chr_P_sfGFP_100x_ref1max",20,'20200724','20200725',
#                     5,np.arange(200),False],
#                     [3,"chr_P_GFPmut3_100x_ref1max",50,'20200724','20200725',
#                     5,np.arange(200),False]]
# files_to_analyse = [[16,"chr_P_sfGFP_40x_ref1max",10,'20200724','20200725',
#                     5,np.arange(200),False],
#                     [6,"chr_P_GFPmut3_40x_ref1max",0,'20200724','20200725',
#                     5,np.arange(200),False]]
# files_to_analyse = [[15,"chr_P_sfGFP_20x_ref10max",0,'20200724','20200727',
#                     5,np.arange(200),False],
#                     [4,"chr_P_GFPmut3_20x_ref10max",0,'20200724','20200727',
#                     5,np.arange(200),False]]

# files_to_analyse = [[14,"chr_P_sfGFP_100x_ref5max",20,'20200729','20200803',5,'all',False],
#                     [3,"chr_P_GFPmut3_100x_ref5max",50,'20200729','20200803',5,'all',False],
#                     [16,"chr_P_sfGFP_40x_ref5max",10,'20200729','20200803',5,np.arange(200),False],
#                     [6,"chr_P_GFPmut3_40x_ref5max",0,'20200729','20200803',5,np.arange(200),False],
#                    [15,"chr_P_sfGFP_20x_ref5max",0,'20200729','20200803',5,np.arange(200),False],
#                     [4,"chr_P_GFPmut3_20x_ref5max",0,'20200729','20200803',5,np.arange(200),False],
#                    [14,"chr_P_sfGFP_100x_ref10max",20,'20200729','20200803',10,'all',False],
#                     [3,"chr_P_GFPmut3_100x_ref10max",50,'20200729','20200803',10,'all',False],
#                     [16,"chr_P_sfGFP_40x_ref10max",10,'20200729','20200803',10,np.arange(100),False],
#                     [6,"chr_P_GFPmut3_40x_ref10max",0,'20200729','20200803',10,np.arange(100),False],
#                    [15,"chr_P_sfGFP_20x_ref10max",0,'20200729','20200803',10,np.arange(100),False],
#                     [4,"chr_P_GFPmut3_20x_ref10max",0,'20200729','20200803',10,np.arange(100),False],
#                     [14,"chr_P_sfGFP_100x_ref1max",20,'20200729','20200803',1,'all',False],
#                     [3,"chr_P_GFPmut3_100x_ref1max",50,'20200729','20200803',1,'all',False],
#                     [16,"chr_P_sfGFP_40x_ref1max",10,'20200729','20200803',1,'all',False],
#                     [6,"chr_P_GFPmut3_40x_ref1max",0,'20200729','20200803',1,'all',False],
#                    [15,"chr_P_sfGFP_20x_ref1max",0,'20200729','20200803',1,'all',False],
#                     [4,"chr_P_GFPmut3_20x_ref1max",0,'20200729','20200803',1,'all',False]]

# generate_analysis_plots(files_to_analyse)

# files_to_analyse = [[38,"plas_P_DHL_sfGFP_100x_ref1max",78,'20200729','20200729',1,'all',False],
#                    [40,"plas_P_DHL_sfGFP_40x_ref1max",15,'20200729','20200729',1,'all',False],
#                    [39,"plas_P_DHL_sfGFP_20x_ref1max",0,'20200729','20200729',1,'all',False],
#                    [44,"plas_X_DHL_GFPmut3_100x_ref1max",68,'20200729','20200729',1,'all',False],
#                    [46,"plas_X_DHL_GFPmut3_40x_ref1max",23,'20200729','20200729',1,'all',False],
#                    [45,"plas_X_DHL_GFPmut3_20x_ref1max",6,'20200729','20200729',1,'all',False]]

# files_to_analyse = [[44,"plas_X_DHL_GFPmut3_100x_ref1max",68,'20200730','20200730',1,'all',False],
#                    [46,"plas_X_DHL_GFPmut3_40x_ref1max",23,'20200730','20200730',1,np.arange(200),False],
#                    [45,"plas_X_DHL_GFPmut3_20x_ref1max",6,'20200730','20200730',1,np.arange(200),False]]

files_to_analyse = [
    [8, "chr_P_Venus_100x_ref5max", 48, "20200803", "20200803", 5, "all", False],
    [0, "chr_P_Dendra2_100x_ref5max", 44, "20200803", "20200803", 5, "all", False],
    [10, "chr_P_Venus_40x_ref5max", 0, "20200803", "20200803", 5, "all", False],
    [
        2,
        "chr_P_Dendra2_40x_ref5max",
        2,
        "20200803",
        "20200803",
        5,
        np.arange(200),
        False,
    ],
    [9, "chr_P_Venus_20x_ref5max", 8, "20200803", "20200803", 5, np.arange(200), False],
    [
        1,
        "chr_P_Dendra2_20x_ref5max",
        0,
        "20200803",
        "20200803",
        5,
        np.arange(200),
        False,
    ],
    [8, "chr_P_Venus_100x_ref10max", 48, "20200803", "20200803", 10, "all", False],
    [0, "chr_P_Dendra2_100x_ref10max", 44, "20200803", "20200803", 10, "all", False],
    [
        10,
        "chr_P_Venus_40x_ref10max",
        0,
        "20200803",
        "20200803",
        10,
        np.arange(100),
        False,
    ],
    [
        2,
        "chr_P_Dendra2_40x_ref10max",
        2,
        "20200803",
        "20200803",
        10,
        np.arange(100),
        False,
    ],
    [
        9,
        "chr_P_Venus_20x_ref10max",
        8,
        "20200803",
        "20200803",
        10,
        np.arange(100),
        False,
    ],
    [
        1,
        "chr_P_Dendra2_20x_ref10max",
        0,
        "20200803",
        "20200803",
        10,
        np.arange(100),
        False,
    ],
    [8, "chr_P_Venus_100x_ref1max", 48, "20200803", "20200803", 1, "all", False],
    [0, "chr_P_Dendra2_100x_ref1max", 44, "20200803", "20200803", 1, "all", False],
    [10, "chr_P_Venus_40x_ref1max", 0, "20200803", "20200803", 1, "all", False],
    [2, "chr_P_Dendra2_40x_ref1max", 2, "20200803", "20200803", 1, "all", False],
    [9, "chr_P_Venus_20x_ref1max", 8, "20200803", "20200803", 1, "all", False],
    [1, "chr_P_Dendra2_20x_ref1max", 0, "20200803", "20200803", 1, "all", False],
]

generate_analysis_plots(files_to_analyse)

In [ ]:
def generate_test_image(
    size=(200, 200),
    mask=None,
    shapes=[["circle", [100, 100], 10, 100]],
    savename="test",
    background=None,
    cell_background=None,
):
    # shape = ['rings'=concentric circles, 'circle'=circle, 'gaussian'=2d gaussian]
    # shape == 'circle': ['circle',centre position indices,radius,intensity]
    # shape == 'rings': ['rings',centre position indices,
    # radius of smallest circle, intensity, width of rings, distance between rings]
    # shape == 'gaussian': ['gaussian', centre position indices, stdevs, intensity] (mean = position)
    # mask should be False where the cell is.
    if not background:
        image = np.zeros(size)
    elif background[0] == "gaussian":
        mean, stdev = background[1:]
        image = np.random.normal(loc=mean, scale=stdev, size=size)
    if np.size(mask) > 1:
        image = np.ma.array(image, mask=mask)
        if cell_background:
            if cell_background[0] == "gaussian":
                mean, stdev = cell_background[1:]
                cell_noise = np.random.normal(loc=mean, scale=stdev, size=size)
                image += cell_noise
    for shape in shapes:
        if shape[0] == "circle" or shape[0] == "rings":
            image_to_add = np.zeros(size)
            position, r, intensity = shape[1:]
            ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
            rel_points = np.where(
                (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
            )
            points = [
                rel_points[0] + position[0] - np.ceil(r),
                rel_points[1] + position[1] - np.ceil(r),
            ]
            image_to_add[
                list(map(int, points[0])), list(map(int, points[1]))
            ] = intensity
        #         if shape[0] == 'rings':

        elif shape[0] == "gaussian":
            position, stdevs, intensity = shape[1:]
            vals0, vals1 = np.meshgrid(np.arange(size[0]), np.arange(size[1]))
            image_to_add = np.exp(
                -(
                    (vals0 - position[0]) ** 2 / (2 * stdevs[0]) ** 2
                    + (vals1 - position[1]) ** 2 / (2 * stdevs[1]) ** 2
                )
            )
            image_to_add *= intensity
        image += image_to_add
    if savename:
        np.savetxt(savename, image)
    return image

In [ ]:
file = filenames[14]
print("file", file)
nd2 = nd2reader.ND2Reader(file)
fov_index = 20
phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
# phase_img = nd2.get_frame_2D(v=0, c=0, z=0)[1100:3000,500:2250]
# fluor_img = nd2.get_frame_2D(v=0, c=1, z=0)[1100:3000,500:2250]
fig, axes = plt.subplots(1, 5, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img)
ax[1].imshow(fluor_img, cmap="RdGy")
seg = segment_phase(phase_img)
ax[2].imshow(seg)
ax[3].imshow(seg[700:850, 425:575])

In [ ]:
mask = np.genfromtxt("cellmask")
mask = np.ma.masked_array(mask, np.logical_not(mask)).mask
ax[4].imshow(mask)

In [ ]:
stdevs = []
rdf_fracs = []

base_name = "20200731/refpoint1_bg100stdev25_stdev2_int100_2blobs"
# testing = np.linspace(1,500,50)
# testing = np.linspace(0.01,20,50)
# testing = np.linspace(0,100,50)
testing = np.arange(45)
test_param = "distance from centre"

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
ax = axes.ravel()

for value in testing:
    test_image = generate_test_image(
        size=(150, 150),
        shapes=[
            ["gaussian", [70 - value, 70], [2, 2], 100],
            ["gaussian", [70 + value, 70], [2, 2], 100],
        ],
        background=["gaussian", 100, 25],
        savename=None,
        mask=mask,
        cell_background=None,
    )
    r_vals, gr_vals = rdf(
        np.genfromtxt("cellmask"),
        test_image,
        ref_point_val=["max", 1],
        cell_indices=[20],
        time_allowed=80,
        savetxt=None,
        start_r=1,
        dr=1,
        intensity_weighted=True,
        stdevs=True,
        stdevs_savetxt="20200731/stdevtemp",
    )
    r_vals, gr_vals = np.array(r_vals[0]), np.array(gr_vals[0])
    rdf_fracs.append(
        area_under(
            r_vals,
            gr_vals,
            0,
            shift=1,
            as_frac_total=True,
            below_thresh_zero=True,
            as_frac_total_original=True,
        )
    )
    stdevs.append(float(np.genfromtxt("20200731/stdevtemp")))
    np.savetxt(base_name + "_rdffracs", rdf_fracs)
    np.savetxt(base_name + "_stdevs", stdevs)
    if value == testing[0]:
        ax[1].imshow(test_image)
        ax[1].set_title(test_param + " = " + "{:.2f}".format(testing[0]))
    elif value == testing[16]:
        ax[2].imshow(test_image)
        ax[2].set_title(test_param + " = " + "{:.2f}".format(testing[16]))
    elif value == testing[30]:
        ax[3].imshow(test_image)
        ax[3].set_title(test_param + " = " + "{:.2f}".format(testing[30]))
    elif value == testing[-1]:
        ax[4].imshow(test_image)
        ax[4].set_title(test_param + " = " + "{:.2f}".format(testing[-1]))

for_colorbar = ax[0].scatter(stdevs, rdf_fracs, c=testing, cmap="viridis")
ax[0].set_xlabel("Stdevs of cell intensity")
ax[0].set_ylabel("Initial integral RDF-1>0 as frac total")
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_ylim(ymin=5e-3, ymax=1)
fig.suptitle(base_name[9:], fontsize=14, y=1.05)
plt.colorbar(for_colorbar, ax=ax[0])
plt.tight_layout()
fig.savefig(base_name[:9] + "/testplot_" + base_name[9:], bbox_inches="tight")

In [ ]:
# Now determine the values of these metrics in different test cases, to check that we get values that correspond
# with what we expect by eye.

# Test intensities (how high above background noise) and stdevs (how diffuse) of centred blobs. Then repeat for
# blobs near the edge.

# Looks weird? Why is initial int the opposite of initial int as frac? Check the zero_below_thresh etc to be
# certain about what these are computing.

base_name = "20200727/refpoint1_nonoise/refpoint1_nonoise"

val_first_rdf_cross = []
area_under_rdf_shift1_thresh0 = []
area_under_rdf_shift1_thresh0_as_frac_total = []
number_rdf_crosses = []
total_area_under_rdf_shift1_thresh0_as_frac_total = []

intensities = np.linspace(1, 500, 20)
stdevs = np.linspace(0.5, 3, 20)

for intensity in intensities:
    val_first_rdf_cross_toadd = []
    area_under_rdf_shift1_thresh0_toadd = []
    area_under_rdf_shift1_thresh0_as_frac_total_toadd = []
    number_rdf_crosses_toadd = []
    total_area_under_rdf_shift1_thresh0_as_frac_total_toadd = []
    for stdev in stdevs:
        test_image = generate_test_image(
            size=(150, 150),
            shapes=[["gaussian", [70, 70], [stdev, stdev], intensity]],
            background=None,
            savename=None,
            mask=mask,
            cell_background=None,
        )
        #         autocorr(np.genfromtxt('cellmask'), test_image, cell_indices=[20], time_allowed=80,
        #         savetxt=None,ref_point_val = ['max',10],r_values = None,
        #             gr_values = None, intensity_weighted = True,start_r=1,dr=1,
        #             rdfsavetxt=None)
        r_vals, gr_vals = rdf(
            np.genfromtxt("cellmask"),
            test_image,
            ref_point_val=["max", 1],
            cell_indices=[20],
            time_allowed=80,
            savetxt=None,
            start_r=1,
            dr=1,
            intensity_weighted=True,
        )
        r_vals, gr_vals = np.array(r_vals[0]), np.array(gr_vals[0])
        val_first_rdf_cross_toadd.append(
            first_threshold_cross(r_vals, gr_vals, 0, shift=1)
        )
        area_under_rdf_shift1_thresh0_toadd.append(
            area_under(
                r_vals,
                gr_vals,
                0,
                shift=1,
                as_frac_total=False,
                below_thresh_zero=False,
            )
        )
        area_under_rdf_shift1_thresh0_as_frac_total_toadd.append(
            area_under(
                r_vals,
                gr_vals,
                0,
                shift=1,
                as_frac_total=True,
                below_thresh_zero=True,
                as_frac_total_original=True,
            )
        )
        number_rdf_crosses_toadd.append(
            number_threshold_crosses(r_vals, gr_vals, 0, shift=1, rel_to_length=True)
        )
        total_area_under_rdf_shift1_thresh0_as_frac_total_toadd.append(
            area_under(
                r_vals,
                gr_vals,
                0,
                shift=1,
                as_frac_total=True,
                below_thresh_zero=True,
                as_frac_total_original=True,
                total_area_above_thresh=True,
            )
        )
    val_first_rdf_cross.append(val_first_rdf_cross_toadd)
    area_under_rdf_shift1_thresh0.append(area_under_rdf_shift1_thresh0_toadd)
    area_under_rdf_shift1_thresh0_as_frac_total.append(
        area_under_rdf_shift1_thresh0_as_frac_total_toadd
    )
    number_rdf_crosses.append(number_rdf_crosses_toadd)
    total_area_under_rdf_shift1_thresh0_as_frac_total.append(
        total_area_under_rdf_shift1_thresh0_as_frac_total_toadd
    )

np.savetxt(base_name + "_intensities", intensities)
np.savetxt(base_name + "_stdevs", stdevs)
np.savetxt(base_name + "_firstrdfcross", val_first_rdf_cross)
np.savetxt(base_name + "_initialintegral", area_under_rdf_shift1_thresh0)
np.savetxt(
    base_name + "_initialintegralasfrac", area_under_rdf_shift1_thresh0_as_frac_total
)
np.savetxt(base_name + "_rdfcrosses", number_rdf_crosses)
np.savetxt(
    base_name + "_totalintegral", total_area_under_rdf_shift1_thresh0_as_frac_total
)


print("Complete")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(9, 6))
ax = axes.ravel()
ax[0].pcolormesh(stdevs, intensities, val_first_rdf_cross)
ax[0].set_title("First RDF-1=0")
ax[1].pcolormesh(stdevs, intensities, area_under_rdf_shift1_thresh0)
ax[1].set_title("Initial integral RDF-1>0")
ax[2].pcolormesh(stdevs, intensities, area_under_rdf_shift1_thresh0_as_frac_total)
ax[2].set_title("Initial integral RDF-1>0 as frac total")
ax[3].pcolormesh(stdevs, intensities, number_rdf_crosses)
ax[3].set_title("Frequency RDF-1=0")
ax[4].pcolormesh(stdevs, intensities, total_area_under_rdf_shift1_thresh0_as_frac_total)
ax[4].set_title("Total integral RDF-1>0")
for axis in ax:
    axis.set_xlabel("Stdev")
    axis.set_ylabel("Intensity")
plt.tight_layout()
plt.suptitle("refpoint1 no noise", y=1.05, fontsize=14)
fig.savefig("20200727/refpoint1_nonoise.png", bbox_inches="tight")

In [ ]:
# 20200731

r, gr, pixels = rdf(
    seg,
    fluor_img,
    cell_indices=np.arange(100),
    ref_point_val=["max", 1],
    dr=1,
    intensity_weighted=True,
    savetxt=None,
)
int_rdf = []
int_rdf_volfactor = []

for i, cell in enumerate(pixels):
    print(i)
    int_rdf.append(np.trapz(gr[i], x=r[i]))
    sum_gr.append(np.sum(gr[i]))
    int_rdf_volfactor.append(np.trapz(gr[i] * np.pi * (2 * np.array(r[i]) + 1), x=r[i]))

values = [int_rdf, int_rdf_volfactor, pixels]
names = ["int_rdf", "int_rdf_volfactor", "pixels"]

combs = itertools.combinations([0, 1, 2], 2)
fig, axes = plt.subplots(1, 3, figsize=(9, 3))
ax = axes.ravel()
for i, comb in enumerate(combs):
    ax[i].scatter(values[comb[0]], values[comb[1]])
    ax[i].set_xlabel(names[comb[0]])
    ax[i].set_ylabel(names[comb[1]])

fig.savefig("20200731/integralrdf_sfGFP100xfov20_100cellsref1")

In [ ]:
plt.imshow(seg[700:850, 425:575])
print(np.amax(seg[700:850, 425:575]))

In [ ]:
# rdf(seg[700:850,425:575], test_image, ref_point_val = 'max', cell_indices=[20], time_allowed=80,
#         savetxt='20200714',start_r=1,dr=1, intensity_weighted = True)
%run Testcreator.ipynb
name = "randomblobs"

autocorr(
    np.genfromtxt("cellmask"),
    test_image,
    cell_indices=[20],
    time_allowed=80,
    savetxt="20200714/gr_as_Y/autocorr_" + name,
    ref_point_val="max",
    r_values=None,
    gr_values=None,
    intensity_weighted=True,
    start_r=1,
    dr=1,
    rdfsavetxt="20200714/gr_as_Y/rdf_" + name,
)

fig, axes = plt.subplots(1, 4, figsize=(12, 3))
ax = axes.ravel()

ax[0].imshow(test_image.data)
ax[0].set_title("fluor_img")

ax[1].imshow(np.ma.masked_where(seg[700:850, 425:575] == 0, test_image))
ax[1].set_title("segmented")

plot_gr_from_file(
    "20200714/gr_as_Y/autocorr_" + name, 1, savefig=None, line=[0], ax=ax[2]
)
ax[2].set_title("autocorr")

plot_gr_from_file("20200714/gr_as_Y/rdf_" + name, 1, savefig=None, line=[1], ax=ax[3])
ax[3].set_title("rdf")

fig.suptitle(name, fontsize=14, y=1.05)
plt.tight_layout()
fig.savefig("20200714/gr_as_Y/plots_" + name, y=1)
np.savetxt("cellmask", seg[700:850, 425:575])

In [ ]:
k_vals, autocorr = autocorr(
    seg,
    fluor_img,
    cell_indices="all",
    time_allowed=80,
    savetxt="20200713autocorr",
    ref_point_val="max",
    r_values=None,
    gr_values=None,
    intensity_weighted=True,
    start_r=1,
    dr=1,
    rdfsavetxt="20200713autocorrrdf",
)

In [ ]:
plot_gr_from_file("20200713autocorr", 1, savefig="20200713autocorr", line_1=False)

In [ ]:
# Plot grs from file
# fig,axes = plt.subplots(2,2,figsize=(20,20))
# ax = axes.ravel()
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_foci",
    indices_to_plot=[1, 2, 11, 25, 27, 28, 33, 52],
    legend=True,
    title="Foci",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_maybefoci",
    indices_to_plot=[5, 8, 14, 20, 22, 30, 32, 40, 55],
    legend=True,
    title="Maybe foci",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_nofoci",
    indices_to_plot=[0, 3, 4, 6, 9, 13, 23, 29, 31, 36, 37, 44, 51, 58],
    legend=True,
    title="No foci",
)

In [ ]:
rough_sqrt = int(np.ceil(np.sqrt(62)))
fig, axes = plt.subplots(rough_sqrt, rough_sqrt, figsize=(25, 25))
ax = axes.ravel()
i = 0
for cell_index in range(62):
    cell_indices = np.where(seg == cell_index + 1)
    cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
    #     fig, ax = plt.subplots(figsize=(5,5))
    ax[i].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[i].set_title(cell_index)
    i += 1
plt.tight_layout()
fig.savefig("20200626/sfGFP_100x_fov20/cells")

In [ ]:
# To check:
#     single point vs gradual gradient (i.e. size of the blob)
#     gradient of the intensity of the blob (i.e. one clear blob and no background or more diffuse blob)
#     different cell shape
#     position of blob within cell
#     intensity of the blob vs background of cell
# Test on 2d Gaussian and hat function